# Example Uses Of “`%%script`” Magic #

The `%%script` cell magic allows the invocation of any number of external languages without the need for installing any custom Jupyter kernels. The drawback is that there is no context maintained between one cell invocation and the next. Another drawback is that the notebook interface insists on syntax-colouring the cell contents as though it were Python code.

The syntax is

> `%%script` _commandline_

Simple example: bash

In [1]:
%%script bash
echo 'hi there!'

hi there!


Use with SBCL:

In [2]:
%%script sbcl
(+ 2 2)

This is SBCL 2.0.1.debian, an implementation of ANSI Common Lisp.
More information about SBCL is available at <http://www.sbcl.org/>.

SBCL is free software, provided as is, with absolutely no warranty.
It is mostly in the public domain; some portions are provided under
BSD-style licenses.  See the CREDITS and COPYING files in the
distribution for more information.
* 4
* 

You probably want to turn off the startup banner message, and the “\*” characters from the default prompting for input by the REPL. The `--script` option does this, but it also turns off automatic output of expression values:

In [3]:
%%script sbcl --script
(+ 2 2) ; produces no output
(princ (+ 2 2)) ; need to do explicit output

4

This may be OK for running functional scripts, but it is not so convenient for simple experimentation.

Alternatively, the `--noinform` option turns off the startup banner. You can then change the prompt to the empty string by setting the SBCL-specific `sb-aclrepl:*prompt*` variable. To access this, you need to `require` the `sb-aclrepl` package. To avoid spurious output, this can all be done with `--eval` options on the command line:

In [4]:
%%script sbcl --noinform --eval "(require 'sb-aclrepl)"  --eval "(setq sb-aclrepl:*prompt* \"\")"
(+ 2 2)

4


Just to make clear the fact that function names and variable names exist in separate name spaces: in the following example, “+” is used as the name of a variable _and_ to refer to the built-in addition function.

In [5]:
%%script sbcl --script
(setf + 2)
(princ (+ + +))

4

Note that access to command-line arguments is not a standard feature of Common Lisp. SBCL provides an extension to do this:

In [6]:
%%script sbcl --script /dev/stdin the quick brown fox
(print sb-ext:*posix-argv*)


("sbcl" "the" "quick" "brown" "fox") 

My example of computing the first _n_ terms of a Fibonacci series:

In [7]:
%%script sbcl --script
(defvar n 15) ; change as required
(do
	(
		(a 1 (+ a b))
		(b 1 a)
		(i 0 (+ i 1))
	)
	((>= i n))

	(format *standard-output* "~A~%" a)
) ; do


1
2
3
5
8
13
21
34
55
89
144
233
377
610
987


Jake’s example of lexical binding in action:

In [8]:
%%script sbcl --noinform --eval "(require 'sb-aclrepl)" --eval "(setq sb-aclrepl:*prompt* \"\")"

(setf (symbol-function 'counter)
    (let ((count 0))
        (lambda ()
             (if (> count 5)
                (setf count 0)
                (incf count)
             ) ; if
        ) ; lambda
    ) ; let
)

(counter)
(counter)
(counter)

#<CLOSURE (LAMBDA ()) {100282599B}>
1
2
3


A more elaborate `%%script` example: compiling and running an entire C program in a cell!

In [9]:
%%script bash
set -e

WORKDIR=$(mktemp -d)
cd "$WORKDIR"

cat >test.c <<EOD
#include <stdio.h>

int main(int argc, char **argv)
  {
    fputs("hello, world!\n", stdout);
  } /*main*/
EOD

gcc -o test test.c

./test

cd
rm -rfv "$WORKDIR"

hello, world!
removed '/tmp/tmp.Tj1kFiTPJG/test.c'
removed '/tmp/tmp.Tj1kFiTPJG/test'
removed directory '/tmp/tmp.Tj1kFiTPJG'


See also: [this list](https://ipython.readthedocs.io/en/stable/interactive/magics.html) of standard magic commands.